# Clustering Assignment: Reproducible Experimental Pipeline

This notebook implements an end-to-end clustering workflow designed for *comparative evaluation* across two dataset variants (**no augmentation** vs. **augmented**). The analysis emphasizes (i) transparent preprocessing, (ii) principled hyperparameter selection, and (iii) quantitative evaluation using internal validity (e.g., silhouette) and external agreement metrics (e.g., purity).

**In the next cell, a minimal sanity check is executed to confirm that the notebook runs correctly before importing the full scientific stack.**

In [ ]:
print("Salam Alikom Habibi")

## 1. Library Imports and Runtime Utilities

In this section, all required scientific-computing and machine-learning dependencies are imported (NumPy/Pandas/Matplotlib and scikit-learn). Optional notebook-only display helpers are also activated to improve readability when presenting intermediate tables and figures.

In [ ]:
from __future__ import annotations

from pathlib import Path
import json
import shutil
import datetime as dt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib.lines import Line2D
from matplotlib.colors import ListedColormap

from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score

from sklearn.cluster import KMeans, DBSCAN, OPTICS

# Optional: pretty side-by-side display inside notebooks
try:
    from IPython.display import display, HTML
    IPY_OK = True
except Exception:
    IPY_OK = False


## 2. Experimental Configuration and Reproducibility Controls

Here, file-system paths (data and output directories), dataset *variants*, random seeds, and algorithm-specific hyperparameters are declared. Centralizing these settings supports reproducibility and facilitates systematic comparisons across clustering methods and preprocessing choices.

In [ ]:
# =========================
# CONFIG
# =========================
ROOT = Path(".").resolve()
DATA_DIR = ROOT / "data"

RUN_TS = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
RUN_DIR = ROOT/"outputs"/ f"outputs_{RUN_TS}"
EXPORT_DIR = RUN_DIR / "csv_exports"

KINDS = ["no_augmentation", "augmented"]
RANDOM_SEED = 42

# ---- Step1 (preprocess / optimized features) ----
DROP_HIGH_CORR = True
CORR_THR = 0.99

USE_PCA = True
PCA_VAR = 0.90          # variance ratio target (e.g. 0.90)
PCA_WHITEN = False
ROW_L2_NORM = False

# ---- Step2 (KMeans) ----
K_RANGE = list(range(2, 10))
SIL_SAMPLE_SIZE = 1000

# ---- Step3 (DBSCAN) ----
DBSCAN_MIN_SAMPLES_LIST = [5, 10, 15, 20]
DBSCAN_EPS_PERCENTILES = [60, 70, 80, 85, 90, 92, 95]
TARGET_K = 4
DBSCAN_MAX_NOISE_RATIO = 0.35

# ---- Step4 (OPTICS) ----
OPTICS_MIN_SAMPLES_LIST = [5, 10, 15, 20]
OPTICS_XI_LIST = [0.03, 0.05, 0.08, 0.10]
OPTICS_MIN_CLUSTER_SIZE_LIST = [0.03, 0.05, 0.08]
OPTICS_MAX_NOISE_RATIO = 0.40

# Create run folders
RUN_DIR.mkdir(parents=True, exist_ok=True)
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

print("[OK] RUN_DIR:", RUN_DIR)
print("[OK] DATA_DIR exists?", DATA_DIR.exists())


## 3. Utility Functions: I/O, Metrics, and Visualization Primitives

This block defines reusable helper functions for: (a) robust directory and JSON/CSV I/O, (b) dataset loading from `*.npy` files, (c) evaluation metrics (including silhouette computations adapted for noise-aware clusterings), and (d) visualization utilities (e.g., PCA-based 2D projections for qualitative inspection).

In [ ]:
# =========================
# UTILS
# =========================
def ensure_dirs(*paths: Path) -> None:
    for p in paths:
        p.mkdir(parents=True, exist_ok=True)


def save_json(obj, path: Path) -> None:
    ensure_dirs(path.parent)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)


def side_by_side_tables(df_left: pd.DataFrame, df_right: pd.DataFrame,
                        title_left: str = "RAW", title_right: str = "OPT") -> None:
    if not IPY_OK:
        print(title_left)
        print(df_left)
        print("\n" + title_right)
        print(df_right)
        return

    html = f"""
    <div style="display:flex; gap:24px; align-items:flex-start;">
      <div style="flex:1;">
        <h4 style="margin:0 0 8px 0;">{title_left}</h4>
        {df_left.to_html(index=False)}
      </div>
      <div style="flex:1;">
        <h4 style="margin:0 0 8px 0;">{title_right}</h4>
        {df_right.to_html(index=False)}
      </div>
    </div>
    """
    display(HTML(html))


def load_feature_names(data_dir: Path) -> list[str]:
    p = data_dir / "feature_names.npy"
    if not p.exists():
        raise FileNotFoundError(f"Missing feature_names.npy in: {data_dir}")
    arr = np.load(p, allow_pickle=True)
    return [str(x) for x in arr]


def load_dataset_npy(data_dir: Path, kind: str):
    base = data_dir / kind
    X_train = np.load(base / "X_train.npy")
    X_test = np.load(base / "X_test.npy")
    y_train = np.load(base / "y_train.npy", allow_pickle=True).astype(str)
    y_test = np.load(base / "y_test.npy", allow_pickle=True).astype(str)
    return X_train, X_test, y_train, y_test


def prepare_compare_dirs(step_root: Path, kind: str):
    raw_out = step_root / kind / "raw"
    opt_out = step_root / kind / "optimized"
    cmp_out = step_root / kind / "compare"
    ensure_dirs(raw_out / "tables", raw_out / "figures", raw_out / "meta")
    ensure_dirs(opt_out / "tables", opt_out / "figures", opt_out / "meta")
    ensure_dirs(cmp_out / "tables", cmp_out / "figures", cmp_out / "meta")
    return raw_out, opt_out, cmp_out


def purity_score(cluster_labels: np.ndarray, true_labels: np.ndarray, noise_label: int | None = -1):
    """Purity (overall) + per-cluster table. If noise_label is not None, it will be ignored."""
    df = pd.DataFrame({"cluster": cluster_labels, "true": true_labels})

    if noise_label is not None:
        df = df[df["cluster"] != noise_label]

    total = len(df)
    if total == 0:
        empty = pd.DataFrame(columns=["cluster", "size", "top_label", "top_count", "purity_cluster"])
        return float("nan"), empty

    per_cluster = []
    correct = 0
    for c, g in df.groupby("cluster"):
        vc = g["true"].value_counts()
        top_label = vc.idxmax()
        top_count = int(vc.max())
        size = int(len(g))
        correct += top_count
        per_cluster.append(
            {
                "cluster": int(c),
                "size": size,
                "top_label": str(top_label),
                "top_count": top_count,
                "purity_cluster": float(top_count / size),
            }
        )

    overall = float(correct / total)
    return overall, pd.DataFrame(per_cluster).sort_values("cluster")


def silhouette_excluding_noise(X: np.ndarray, labels: np.ndarray, noise_label: int = -1,
                               sample_size: int | None = None) -> float:
    mask = labels != noise_label
    labs = labels[mask]
    if mask.sum() < 10 or len(np.unique(labs)) < 2:
        return float("nan")
    X_use = X[mask]
    use_sample_size = sample_size if (sample_size is not None and X_use.shape[0] > sample_size) else None
    return float(silhouette_score(X_use, labs, sample_size=use_sample_size, random_state=RANDOM_SEED))


def choose_best_by_target_k(df_results: pd.DataFrame, target_k: int, max_noise_ratio: float) -> dict | None:
    """Choose best row: prefer n_clusters close to target_k, higher silhouette, lower noise."""
    df = df_results.copy()
    df = df[df["n_clusters"] >= 2]
    if df.empty:
        return None

    if (df["noise_ratio"] <= max_noise_ratio).any():
        df = df[df["noise_ratio"] <= max_noise_ratio]

    df["k_distance"] = (df["n_clusters"] - target_k).abs()
    df = df.sort_values(["k_distance", "silhouette_excl_noise", "noise_ratio"], ascending=[True, False, True])
    best = df.iloc[0].to_dict()
    return best


def pca2_project(X: np.ndarray) -> np.ndarray:
    return PCA(n_components=2, random_state=RANDOM_SEED).fit_transform(X)


def encode_labels_for_cmap(labels: np.ndarray, include_noise: bool = True, noise_label: int = -1):
    labels = np.asarray(labels)
    is_str = labels.dtype.kind in ["U", "S", "O"]

    if is_str:
        uniq = sorted(pd.unique(labels).tolist())
    else:
        uniq = np.unique(labels)
        uniq = np.sort(uniq)
        if include_noise and (noise_label in uniq):
            uniq = np.r_[noise_label, uniq[uniq != noise_label]]

    mapping = {lab: i for i, lab in enumerate(uniq)}
    enc = np.array([mapping[v] for v in labels])
    return enc, uniq, mapping


def plot_pca2_scatter(X: np.ndarray, labels: np.ndarray, title: str, outpath: Path,
                      cmap_name: str = "tab20", include_noise: bool = True, noise_label: int = -1,
                      legend_title: str = "Labels", show: bool = False) -> None:
    X2 = pca2_project(X)
    enc, uniq, mapping = encode_labels_for_cmap(labels, include_noise=include_noise, noise_label=noise_label)

    base = plt.cm.get_cmap(cmap_name, len(uniq))
    colors = [base(i) for i in range(len(uniq))]
    cmap = ListedColormap(colors)

    plt.figure(figsize=(6, 4))
    plt.scatter(X2[:, 0], X2[:, 1], s=10, alpha=0.8, c=enc, cmap=cmap)
    plt.title(title)
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.grid(True, alpha=0.2)

    handles = []
    for lab in uniq:
        if not (np.asarray(labels).dtype.kind in ["U", "S", "O"]) and lab == noise_label:
            name = "Noise (-1)"
        else:
            name = str(lab) if (np.asarray(labels).dtype.kind in ["U", "S", "O"]) else f"Cluster {int(lab)}"

        handles.append(
            Line2D([0], [0], marker="o", linestyle="", markerfacecolor=cmap(mapping[lab]),
                   markeredgecolor="none", markersize=6, label=name)
        )
    plt.legend(handles=handles, title=legend_title, loc="best", frameon=True)

    plt.tight_layout()
    ensure_dirs(outpath.parent)
    plt.savefig(outpath, dpi=200)
    if show:
        plt.show()
    plt.close()


def plot_pca2_side_by_side_truth_vs_cluster(X: np.ndarray, y_true: np.ndarray, cluster_labels: np.ndarray,
                                           title: str, outpath: Path,
                                           include_noise: bool = True, noise_label: int = -1,
                                           cmap_true: str = "Set1", cmap_cluster: str = "Set2",
                                           show: bool = True) -> None:
    X2 = pca2_project(X)

    y_enc, uniq_true, map_true = encode_labels_for_cmap(y_true, include_noise=False, noise_label=noise_label)
    c_enc, uniq_cl, map_cl = encode_labels_for_cmap(cluster_labels, include_noise=include_noise, noise_label=noise_label)

    cmapT = plt.cm.get_cmap(cmap_true, len(uniq_true))
    cmapC = plt.cm.get_cmap(cmap_cluster, len(uniq_cl))

    fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharex=True, sharey=True)

    axes[0].scatter(X2[:, 0], X2[:, 1], s=10, alpha=0.8, c=y_enc, cmap=cmapT)
    axes[0].set_title("TRUE (language)")
    axes[0].grid(True, alpha=0.2)
    true_handles = [
        Line2D([0], [0], marker="o", linestyle="", markerfacecolor=cmapT(i), markeredgecolor="none",
               markersize=6, label=str(lab))
        for i, lab in enumerate(uniq_true)
    ]
    axes[0].legend(handles=true_handles, title="Language", loc="best", frameon=True)

    axes[1].scatter(X2[:, 0], X2[:, 1], s=10, alpha=0.8, c=c_enc, cmap=cmapC)
    axes[1].set_title("CLUSTER")
    axes[1].grid(True, alpha=0.2)

    cl_handles = []
    for i, lab in enumerate(uniq_cl):
        if lab == noise_label:
            name = "Noise (-1)"
        else:
            name = f"Cluster {int(lab)}"
        cl_handles.append(
            Line2D([0], [0], marker="o", linestyle="", markerfacecolor=cmapC(i), markeredgecolor="none",
                   markersize=6, label=name)
        )
    axes[1].legend(handles=cl_handles, title="Clusters", loc="best", frameon=True)

    fig.suptitle(title, y=1.02)
    fig.tight_layout()
    ensure_dirs(outpath.parent)
    fig.savefig(outpath, dpi=200)
    if show:
        plt.show()
    plt.close(fig)


## 4. Optional CSV Export for Reporting and Manual Inspection

This section exports feature names and (when available) metadata to CSV, and serializes the train/test matrices to tabular form. Such exports are useful for debugging, external reporting, and interoperability with non-Python tooling.

In [ ]:
def step0_export_to_csv() -> Path:
    feature_names = load_feature_names(DATA_DIR)

    pd.DataFrame({"feature": feature_names}).to_csv(EXPORT_DIR / "feature_names.csv", index=False)
    print("[SAVED] feature_names.csv ->", EXPORT_DIR / "feature_names.csv")

    md_path = DATA_DIR / "metadata.csv"
    if md_path.exists():
        md = pd.read_csv(md_path)
        md.to_csv(EXPORT_DIR / "metadata.csv", index=False)
        print("[SAVED] metadata.csv ->", EXPORT_DIR / "metadata.csv")
    else:
        print("[WARN] metadata.csv پیدا نشد؛ از این بخش رد شدم.")

    for kind in KINDS:
        base = DATA_DIR / kind
        if not base.exists():
            print(f"[WARN] پوشه داده‌ها برای {kind} وجود ندارد: {base} (skip)")
            continue

        X_train, X_test, y_train, y_test = load_dataset_npy(DATA_DIR, kind)

        df_X_train = pd.DataFrame(X_train, columns=feature_names)
        df_X_test = pd.DataFrame(X_test, columns=feature_names)

        kind_out = EXPORT_DIR / kind
        ensure_dirs(kind_out)

        df_X_train.to_csv(kind_out / "X_train.csv", index=False)
        df_X_test.to_csv(kind_out / "X_test.csv", index=False)
        pd.DataFrame({"label": y_train}).to_csv(kind_out / "y_train.csv", index=False)
        pd.DataFrame({"label": y_test}).to_csv(kind_out / "y_test.csv", index=False)

        df_train = df_X_train.copy()
        df_train["label"] = y_train
        df_test = df_X_test.copy()
        df_test["label"] = y_test

        df_train.to_csv(kind_out / "train_with_label.csv", index=False)
        df_test.to_csv(kind_out / "test_with_label.csv", index=False)

        print(f"[SAVED] CSV export for kind={kind} -> {kind_out}")

    print("[DONE] Step0 finished. EXPORT_DIR:", EXPORT_DIR)
    return EXPORT_DIR


# Run step0 (uncomment if needed)
step0_export_to_csv()


## 5. Step 1 — Preprocessing and Feature-Space Optimization

This block implements the preprocessing stage that prepares both **raw** and **optimized** feature representations. The optimization may include removing highly correlated features, optional PCA (variance-retaining dimensionality reduction), and optional row-wise normalization; the resulting artifacts are saved for downstream clustering experiments.

In [ ]:
def drop_high_corr(X_train: np.ndarray, X_test: np.ndarray, feature_names: list[str], thr: float):
    corr = np.corrcoef(X_train, rowvar=False)
    to_drop = set()

    for i in range(corr.shape[0]):
        if i in to_drop:
            continue
        for j in range(i + 1, corr.shape[0]):
            if abs(corr[i, j]) > thr:
                to_drop.add(j)

    to_drop = sorted(to_drop)
    kept_mask = np.ones(X_train.shape[1], dtype=bool)
    kept_mask[to_drop] = False
    kept_idx = np.where(kept_mask)[0]

    dropped_features = [feature_names[i] for i in to_drop]
    kept_features = [feature_names[i] for i in kept_idx]

    return (
        X_train[:, kept_mask],
        X_test[:, kept_mask],
        kept_idx,
        kept_features,
        dropped_features,
    )


def step1_prepare_raw_and_optimized() -> Path:
    out_root = RUN_DIR / "step1_raw_and_optimized_data"
    ensure_dirs(out_root)

    feature_names = load_feature_names(DATA_DIR)
    pd.DataFrame({"feature": feature_names}).to_csv(out_root / "feature_names.csv", index=False)

    for kind in KINDS:
        print("\n" + "=" * 90)
        print("[KIND]", kind)

        X_train_raw, X_test_raw, y_train, y_test = load_dataset_npy(DATA_DIR, kind)

        kind_root = out_root / kind
        if kind_root.exists():
            shutil.rmtree(kind_root)
        ensure_dirs(kind_root)

        raw_dir = kind_root / "raw"
        opt_dir = kind_root / "optimized"
        meta_dir = kind_root / "meta"
        ensure_dirs(raw_dir, opt_dir, meta_dir)

        # ---- save RAW (csv + npy) ----
        pd.DataFrame(X_train_raw, columns=feature_names).to_csv(raw_dir / "X_train.csv", index=False)
        pd.DataFrame(X_test_raw, columns=feature_names).to_csv(raw_dir / "X_test.csv", index=False)
        pd.DataFrame({"label": y_train}).to_csv(raw_dir / "y_train.csv", index=False)
        pd.DataFrame({"label": y_test}).to_csv(raw_dir / "y_test.csv", index=False)

        np.save(raw_dir / "X_train.npy", X_train_raw)
        np.save(raw_dir / "X_test.npy", X_test_raw)
        np.save(raw_dir / "y_train.npy", y_train)
        np.save(raw_dir / "y_test.npy", y_test)

        # ---- build OPT ----
        X_train_opt = X_train_raw.copy()
        X_test_opt = X_test_raw.copy()
        current_feature_names = feature_names[:]

        report = {
            "kind": kind,
            "input_shape_train": list(X_train_raw.shape),
            "input_shape_test": list(X_test_raw.shape),
            "ops": [],
        }

        if DROP_HIGH_CORR:
            X_train_opt, X_test_opt, kept_idx, kept_features, dropped_features = drop_high_corr(
                X_train_opt, X_test_opt, current_feature_names, thr=CORR_THR
            )
            current_feature_names = kept_features

            np.save(meta_dir / "kept_feature_indices.npy", kept_idx)
            pd.DataFrame({"kept_feature": kept_features}).to_csv(meta_dir / "kept_feature_names.csv", index=False)
            pd.DataFrame({"dropped_feature": dropped_features}).to_csv(meta_dir / "dropped_feature_names.csv", index=False)

            report["ops"].append(
                {
                    "name": "drop_high_corr",
                    "corr_thr": CORR_THR,
                    "dropped_count": int(len(dropped_features)),
                    "kept_count": int(len(kept_features)),
                }
            )

        if USE_PCA:
            pca = PCA(n_components=PCA_VAR, whiten=PCA_WHITEN, svd_solver="full")
            X_train_opt = pca.fit_transform(X_train_opt)
            X_test_opt = pca.transform(X_test_opt)

            n_comp = X_train_opt.shape[1]
            current_feature_names = [f"pc_{i+1}" for i in range(n_comp)]

            np.save(meta_dir / "pca_components.npy", pca.components_)
            np.save(meta_dir / "pca_explained_variance_ratio.npy", pca.explained_variance_ratio_)

            report["ops"].append(
                {
                    "name": "pca",
                    "n_components": int(n_comp),
                    "pca_var_target": float(PCA_VAR),
                    "explained_variance_ratio_sum": float(np.sum(pca.explained_variance_ratio_)),
                    "whiten": bool(PCA_WHITEN),
                }
            )

        if ROW_L2_NORM:
            X_train_opt = normalize(X_train_opt, norm="l2", axis=1, copy=True)
            X_test_opt = normalize(X_test_opt, norm="l2", axis=1, copy=True)
            report["ops"].append({"name": "row_l2_normalize"})

        report["output_shape_train"] = list(X_train_opt.shape)
        report["output_shape_test"] = list(X_test_opt.shape)

        # ---- save OPT (csv + npy) ----
        pd.DataFrame({"feature": current_feature_names}).to_csv(opt_dir / "feature_names_opt.csv", index=False)

        pd.DataFrame(X_train_opt, columns=current_feature_names).to_csv(opt_dir / "X_train.csv", index=False)
        pd.DataFrame(X_test_opt, columns=current_feature_names).to_csv(opt_dir / "X_test.csv", index=False)
        pd.DataFrame({"label": y_train}).to_csv(opt_dir / "y_train.csv", index=False)
        pd.DataFrame({"label": y_test}).to_csv(opt_dir / "y_test.csv", index=False)

        np.save(opt_dir / "X_train.npy", X_train_opt)
        np.save(opt_dir / "X_test.npy", X_test_opt)
        np.save(opt_dir / "y_train.npy", y_train)
        np.save(opt_dir / "y_test.npy", y_test)

        save_json(report, meta_dir / "opt_pipeline_report.json")
        print(f"[DONE] {kind} -> RAW:{raw_dir} | OPT:{opt_dir}")

    save_json({"run_dir": str(RUN_DIR), "kinds": KINDS}, out_root / "step1_index.json")
    print("\n[ALL DONE] Step1 outputs:", out_root)
    return out_root


# Run step1
step1_prepare_raw_and_optimized()


## 6. Step 2 — K-Means: Model Selection and Comparative Evaluation

In this section, K-Means is fit over a range of cluster counts and evaluated (e.g., via silhouette score, and agreement-based summaries where labels are available). Results are aggregated and visualized to support a defensible selection of `k` and to compare performance between raw and optimized representations.

In [ ]:
def load_step1_variant(kind: str, variant: str):
    base = RUN_DIR / "step1_raw_and_optimized_data" / kind / variant
    X_train = np.load(base / "X_train.npy")
    y_train = np.load(base / "y_train.npy", allow_pickle=True).astype(str)
    return X_train, y_train


def make_kmeans(k: int) -> KMeans:
    # For compatibility with older sklearn versions (n_init="auto" vs int)
    try:
        return KMeans(n_clusters=k, n_init="auto", random_state=RANDOM_SEED)
    except TypeError:
        return KMeans(n_clusters=k, n_init=10, random_state=RANDOM_SEED)


def kmeans_sweep(X: np.ndarray, k_list: list[int]):
    rows = []
    labels_by_k = {}

    n = X.shape[0]
    sample_size = SIL_SAMPLE_SIZE if n > SIL_SAMPLE_SIZE else None

    for k in k_list:
        km = make_kmeans(k)
        labels = km.fit_predict(X)

        inertia = float(km.inertia_)
        sil = float("nan") if len(np.unique(labels)) < 2 else float(
            silhouette_score(X, labels, sample_size=sample_size, random_state=RANDOM_SEED)
        )

        rows.append({"k": int(k), "inertia": inertia, "silhouette": sil})
        labels_by_k[int(k)] = labels

    df = pd.DataFrame(rows)
    df_valid = df.dropna(subset=["silhouette"])
    best = df_valid.loc[df_valid["silhouette"].idxmax()].to_dict()
    best_k = int(best["k"])
    return df, best, labels_by_k[best_k], {"silhouette_sample_size": sample_size}


def plot_metric_curve(df: pd.DataFrame, ycol: str, title: str, out_png: Path) -> None:
    plt.figure(figsize=(6, 4))
    plt.plot(df["k"], df[ycol], marker="o")
    plt.title(title)
    plt.xlabel("k")
    plt.ylabel(ycol)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    ensure_dirs(out_png.parent)
    plt.savefig(out_png, dpi=200)
    plt.close()


def plot_metric_curve_side_by_side(df_raw: pd.DataFrame, df_opt: pd.DataFrame, ycol: str,
                                  title: str, out_png: Path) -> None:
    fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharey=True)
    axes[0].plot(df_raw["k"], df_raw[ycol], marker="o")
    axes[0].set_title("RAW")
    axes[0].grid(True, alpha=0.3)

    axes[1].plot(df_opt["k"], df_opt[ycol], marker="o")
    axes[1].set_title("OPT")
    axes[1].grid(True, alpha=0.3)

    fig.suptitle(title, y=1.02)
    fig.tight_layout()
    ensure_dirs(out_png.parent)
    fig.savefig(out_png, dpi=200)
    plt.show()
    plt.close(fig)


def step2_kmeans_compare() -> Path:
    out_root = RUN_DIR / "step2_kmeans_compare"
    ensure_dirs(out_root)

    for kind in KINDS:
        print("\n" + "=" * 90)
        print("[KIND]", kind)

        raw_out, opt_out, cmp_out = prepare_compare_dirs(out_root, kind)

        Xr, yr = load_step1_variant(kind, "raw")
        Xo, yo = load_step1_variant(kind, "optimized")

        df_raw, best_raw, best_labels_raw, raw_meta = kmeans_sweep(Xr, K_RANGE)
        df_opt, best_opt, best_labels_opt, opt_meta = kmeans_sweep(Xo, K_RANGE)

        df_raw.to_csv(raw_out / "tables" / "kmeans_metrics.csv", index=False)
        df_opt.to_csv(opt_out / "tables" / "kmeans_metrics.csv", index=False)

        pd.DataFrame({"cluster": best_labels_raw}).to_csv(raw_out / "tables" / "labels_best.csv", index=False)
        pd.DataFrame({"cluster": best_labels_opt}).to_csv(opt_out / "tables" / "labels_best.csv", index=False)

        pur_raw, pur_table_raw = purity_score(best_labels_raw, yr, noise_label=None)
        pur_opt, pur_table_opt = purity_score(best_labels_opt, yo, noise_label=None)

        pur_table_raw.to_csv(raw_out / "tables" / "purity_best.csv", index=False)
        pur_table_opt.to_csv(opt_out / "tables" / "purity_best.csv", index=False)

        save_json({"best": best_raw, **raw_meta, "purity_overall": pur_raw}, raw_out / "meta" / "summary.json")
        save_json({"best": best_opt, **opt_meta, "purity_overall": pur_opt}, opt_out / "meta" / "summary.json")

        print(f"[BEST] RAW k={int(best_raw['k'])} sil={best_raw['silhouette']:.4f} purity={pur_raw:.4f}")
        print(f"[BEST] OPT k={int(best_opt['k'])} sil={best_opt['silhouette']:.4f} purity={pur_opt:.4f}")

        side_by_side_tables(df_raw, df_opt, title_left=f"RAW ({kind})", title_right=f"OPT ({kind})")

        # plots
        plot_metric_curve(df_raw, "inertia", f"Elbow (Inertia) — RAW — {kind}", raw_out / "figures" / "elbow_inertia.png")
        plot_metric_curve(df_opt, "inertia", f"Elbow (Inertia) — OPT — {kind}", opt_out / "figures" / "elbow_inertia.png")
        plot_metric_curve(df_raw, "silhouette", f"Silhouette vs k — RAW — {kind}", raw_out / "figures" / "silhouette.png")
        plot_metric_curve(df_opt, "silhouette", f"Silhouette vs k — OPT — {kind}", opt_out / "figures" / "silhouette.png")

        plot_metric_curve_side_by_side(df_raw, df_opt, "inertia",
                                       f"Elbow (Inertia) side-by-side — {kind}",
                                       cmp_out / "figures" / "elbow_side_by_side.png")
        plot_metric_curve_side_by_side(df_raw, df_opt, "silhouette",
                                       f"Silhouette side-by-side — {kind}",
                                       cmp_out / "figures" / "silhouette_side_by_side.png")

        # PCA2 visuals (true + best clusters)
        plot_pca2_scatter(Xr, yr, f"PCA2 TRUE labels — RAW — {kind}",
                          raw_out / "figures" / "pca2_true_labels.png",
                          legend_title="Language", include_noise=False)
        plot_pca2_scatter(Xo, yo, f"PCA2 TRUE labels — OPT — {kind}",
                          opt_out / "figures" / "pca2_true_labels.png",
                          legend_title="Language", include_noise=False)

        plot_pca2_scatter(Xr, best_labels_raw, f"PCA2 KMeans best — RAW — {kind} (k={int(best_raw['k'])})",
                          raw_out / "figures" / "pca2_kmeans_best.png",
                          legend_title="Clusters", include_noise=False)
        plot_pca2_scatter(Xo, best_labels_opt, f"PCA2 KMeans best — OPT — {kind} (k={int(best_opt['k'])})",
                          opt_out / "figures" / "pca2_kmeans_best.png",
                          legend_title="Clusters", include_noise=False)

        plot_pca2_side_by_side_truth_vs_cluster(
            Xr, yr, best_labels_raw,
            title=f"RAW — {kind} — TRUE vs KMeans (k={int(best_raw['k'])})",
            outpath=cmp_out / "figures" / "raw_truth_vs_kmeans.png",
            include_noise=False,
        )
        plot_pca2_side_by_side_truth_vs_cluster(
            Xo, yo, best_labels_opt,
            title=f"OPT — {kind} — TRUE vs KMeans (k={int(best_opt['k'])})",
            outpath=cmp_out / "figures" / "opt_truth_vs_kmeans.png",
            include_noise=False,
        )

        # combined metrics table
        df_raw2 = df_raw.assign(variant="raw")
        df_opt2 = df_opt.assign(variant="optimized")
        pd.concat([df_raw2, df_opt2], ignore_index=True).to_csv(cmp_out / "tables" / "kmeans_metrics_combined.csv", index=False)

    print("\n[DONE] Step2 completed. Saved under:", out_root)
    return out_root


# Run step2
step2_kmeans_compare()


## 7. Step 3 — DBSCAN: Density-Based Clustering and ε (Epsilon) Selection

This block evaluates DBSCAN under multiple `min_samples` settings and candidate `ε` values derived from the *k-distance* curve. Because DBSCAN can assign **noise points**, evaluation is performed with noise-aware metrics and constraints (e.g., maximum admissible noise ratio), enabling principled selection of feasible configurations.

In [ ]:
def k_distance_curve(X: np.ndarray, k: int) -> np.ndarray:
    nn = NearestNeighbors(n_neighbors=k)
    nn.fit(X)
    dists, _ = nn.kneighbors(X)
    kth = dists[:, -1]
    return np.sort(kth)


def eps_candidates_from_percentiles(kth_sorted: np.ndarray, percentiles: list[int]) -> list[float]:
    return [float(np.percentile(kth_sorted, p)) for p in percentiles]


def dbscan_eval(X: np.ndarray, y_true: np.ndarray, eps: float, min_samples: int):
    model = DBSCAN(eps=float(eps), min_samples=int(min_samples))
    labels = model.fit_predict(X)

    n = len(labels)
    n_noise = int(np.sum(labels == -1))
    noise_ratio = float(n_noise / n)

    clusters = [c for c in np.unique(labels) if c != -1]
    n_clusters = int(len(clusters))

    sil = silhouette_excluding_noise(X, labels, noise_label=-1, sample_size=SIL_SAMPLE_SIZE)
    pur, pur_tbl = purity_score(labels, y_true, noise_label=-1)

    sizes = pd.Series(labels[labels != -1]).value_counts().sort_index()
    min_size = int(sizes.min()) if len(sizes) else 0
    max_size = int(sizes.max()) if len(sizes) else 0

    row = {
        "eps": float(eps),
        "min_samples": int(min_samples),
        "n_clusters": n_clusters,
        "noise_ratio": noise_ratio,
        "silhouette_excl_noise": sil,
        "purity_excl_noise": float(pur),
        "min_cluster_size": min_size,
        "max_cluster_size": max_size,
    }
    return labels, row, pur_tbl


def step3_dbscan_compare() -> Path:
    out_root = RUN_DIR / "step3_dbscan_compare"
    ensure_dirs(out_root)

    for kind in KINDS:
        print("\n" + "=" * 90)
        print("[KIND]", kind)

        raw_out, opt_out, cmp_out = prepare_compare_dirs(out_root, kind)

        Xr, yr = load_step1_variant(kind, "raw")
        Xo, yo = load_step1_variant(kind, "optimized")

        # ---- RAW grid ----
        raw_rows = []
        raw_best = None
        raw_best_labels = None
        raw_best_purity_tbl = None

        for ms in DBSCAN_MIN_SAMPLES_LIST:
            kth_sorted = k_distance_curve(Xr, k=ms)
            eps_list = eps_candidates_from_percentiles(kth_sorted, DBSCAN_EPS_PERCENTILES)

            plt.figure(figsize=(6, 4))
            plt.plot(kth_sorted)
            plt.title(f"k-distance curve (RAW) — {kind} — min_samples={ms}")
            plt.xlabel("points sorted")
            plt.ylabel(f"distance to {ms}-th NN")
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.savefig(raw_out / "figures" / f"kdist_ms_{ms}.png", dpi=200)
            plt.close()

            for eps in eps_list:
                labels, row, pur_tbl = dbscan_eval(Xr, yr, eps=eps, min_samples=ms)
                raw_rows.append(row)

        df_raw = pd.DataFrame(raw_rows).sort_values(["silhouette_excl_noise", "noise_ratio"], ascending=[False, True])
        df_raw.to_csv(raw_out / "tables" / "dbscan_grid_results.csv", index=False)

        raw_best = choose_best_by_target_k(df_raw, target_k=TARGET_K, max_noise_ratio=DBSCAN_MAX_NOISE_RATIO)
        if raw_best is not None:
            raw_best_labels, _, raw_best_purity_tbl = dbscan_eval(
                Xr, yr, eps=raw_best["eps"], min_samples=int(raw_best["min_samples"])
            )
            pd.DataFrame({"cluster": raw_best_labels}).to_csv(raw_out / "tables" / "labels_best.csv", index=False)
            raw_best_purity_tbl.to_csv(raw_out / "tables" / "purity_best.csv", index=False)
            save_json(raw_best, raw_out / "meta" / "best_params.json")

            plot_pca2_scatter(
                Xr, raw_best_labels,
                title=f"DBSCAN BEST (RAW) — {kind} — eps={raw_best['eps']:.4f}, ms={int(raw_best['min_samples'])}",
                outpath=raw_out / "figures" / "pca2_best_clusters.png",
                include_noise=True,
            )
            plot_pca2_side_by_side_truth_vs_cluster(
                Xr, yr, raw_best_labels,
                title=f"RAW — {kind} — TRUE vs DBSCAN (eps={raw_best['eps']:.4f}, ms={int(raw_best['min_samples'])})",
                outpath=cmp_out / "figures" / "raw_truth_vs_dbscan.png",
                include_noise=True,
            )

        # ---- OPT grid ----
        opt_rows = []
        opt_best = None
        opt_best_labels = None
        opt_best_purity_tbl = None

        for ms in DBSCAN_MIN_SAMPLES_LIST:
            kth_sorted = k_distance_curve(Xo, k=ms)
            eps_list = eps_candidates_from_percentiles(kth_sorted, DBSCAN_EPS_PERCENTILES)

            plt.figure(figsize=(6, 4))
            plt.plot(kth_sorted)
            plt.title(f"k-distance curve (OPT) — {kind} — min_samples={ms}")
            plt.xlabel("points sorted")
            plt.ylabel(f"distance to {ms}-th NN")
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.savefig(opt_out / "figures" / f"kdist_ms_{ms}.png", dpi=200)
            plt.close()

            for eps in eps_list:
                labels, row, pur_tbl = dbscan_eval(Xo, yo, eps=eps, min_samples=ms)
                opt_rows.append(row)

        df_opt = pd.DataFrame(opt_rows).sort_values(["silhouette_excl_noise", "noise_ratio"], ascending=[False, True])
        df_opt.to_csv(opt_out / "tables" / "dbscan_grid_results.csv", index=False)

        opt_best = choose_best_by_target_k(df_opt, target_k=TARGET_K, max_noise_ratio=DBSCAN_MAX_NOISE_RATIO)
        if opt_best is not None:
            opt_best_labels, _, opt_best_purity_tbl = dbscan_eval(
                Xo, yo, eps=opt_best["eps"], min_samples=int(opt_best["min_samples"])
            )
            pd.DataFrame({"cluster": opt_best_labels}).to_csv(opt_out / "tables" / "labels_best.csv", index=False)
            opt_best_purity_tbl.to_csv(opt_out / "tables" / "purity_best.csv", index=False)
            save_json(opt_best, opt_out / "meta" / "best_params.json")

            plot_pca2_scatter(
                Xo, opt_best_labels,
                title=f"DBSCAN BEST (OPT) — {kind} — eps={opt_best['eps']:.4f}, ms={int(opt_best['min_samples'])}",
                outpath=opt_out / "figures" / "pca2_best_clusters.png",
                include_noise=True,
            )
            plot_pca2_side_by_side_truth_vs_cluster(
                Xo, yo, opt_best_labels,
                title=f"OPT — {kind} — TRUE vs DBSCAN (eps={opt_best['eps']:.4f}, ms={int(opt_best['min_samples'])})",
                outpath=cmp_out / "figures" / "opt_truth_vs_dbscan.png",
                include_noise=True,
            )

        # ---- Show top-10 tables ----
        top_raw = df_raw.head(10).reset_index(drop=True)
        top_opt = df_opt.head(10).reset_index(drop=True)
        print("[BEST RAW]", raw_best)
        print("[BEST OPT]", opt_best)
        side_by_side_tables(top_raw, top_opt, title_left=f"RAW Top-10 ({kind})", title_right=f"OPT Top-10 ({kind})")

        # ---- one combined figure: k-distance + PCA2 best ----
        ms_show = 10 if 10 in DBSCAN_MIN_SAMPLES_LIST else DBSCAN_MIN_SAMPLES_LIST[0]
        fig, axes = plt.subplots(2, 2, figsize=(12, 8))

        rk = k_distance_curve(Xr, k=ms_show)
        ok = k_distance_curve(Xo, k=ms_show)

        axes[0, 0].plot(rk)
        axes[0, 0].set_title(f"RAW k-distance (ms={ms_show}) — {kind}")
        axes[0, 0].grid(True, alpha=0.3)

        axes[0, 1].plot(ok)
        axes[0, 1].set_title(f"OPT k-distance (ms={ms_show}) — {kind}")
        axes[0, 1].grid(True, alpha=0.3)

        if raw_best_labels is not None:
            X2 = pca2_project(Xr)
            axes[1, 0].scatter(X2[:, 0], X2[:, 1], s=10, alpha=0.8, c=raw_best_labels)
            axes[1, 0].set_title("RAW best clusters (PCA2 view)")
            axes[1, 0].grid(True, alpha=0.2)
        else:
            axes[1, 0].set_title("RAW best clusters (not found)")
            axes[1, 0].axis("off")

        if opt_best_labels is not None:
            X2 = pca2_project(Xo)
            axes[1, 1].scatter(X2[:, 0], X2[:, 1], s=10, alpha=0.8, c=opt_best_labels)
            axes[1, 1].set_title("OPT best clusters (PCA2 view)")
            axes[1, 1].grid(True, alpha=0.2)
        else:
            axes[1, 1].set_title("OPT best clusters (not found)")
            axes[1, 1].axis("off")

        fig.suptitle(f"DBSCAN side-by-side — {kind}", y=1.02)
        fig.tight_layout()
        plt.show()
        fig.savefig(cmp_out / "figures" / "dbscan_side_by_side.png", dpi=200)
        plt.close(fig)

        # ---- save compare summary table ----
        comp = []
        if raw_best is not None:
            comp.append({"variant": "raw", **raw_best})
        if opt_best is not None:
            comp.append({"variant": "optimized", **opt_best})
        pd.DataFrame(comp).to_csv(cmp_out / "tables" / "best_compare.csv", index=False)

    print("\n[DONE] Step3 (DBSCAN) completed. Saved under:", out_root)
    return out_root


# Run step3
step3_dbscan_compare()


## 8. Step 4 — OPTICS: Reachability Analysis and Noise-Aware Evaluation

Here, OPTICS is used as an alternative density-based method that can recover clusters across varying densities. The workflow fits OPTICS over a grid of parameters (e.g., `min_samples`, `xi`, and `min_cluster_size`), produces reachability diagnostics, and evaluates clusterings while accounting for noise assignments.

In [ ]:
def optics_fit_predict(X: np.ndarray, min_samples: int, xi: float, min_cluster_size: float):
    model = OPTICS(
        min_samples=int(min_samples),
        xi=float(xi),
        min_cluster_size=min_cluster_size,
        cluster_method="xi",
    )
    labels = model.fit_predict(X)
    return model, labels


def eval_noise_clustering(X: np.ndarray, y_true: np.ndarray, labels: np.ndarray):
    n = len(labels)
    noise_ratio = float(np.mean(labels == -1))
    n_clusters = int(len([c for c in np.unique(labels) if c != -1]))

    sil = silhouette_excluding_noise(X, labels, noise_label=-1, sample_size=SIL_SAMPLE_SIZE)
    pur, pur_tbl = purity_score(labels, y_true, noise_label=-1)

    return {
        "n_clusters": n_clusters,
        "noise_ratio": noise_ratio,
        "silhouette_excl_noise": sil,
        "purity_excl_noise": float(pur),
    }, pur_tbl


def plot_reachability(model: OPTICS, title: str, outpath: Path, show: bool = False) -> None:
    reachability = model.reachability_[model.ordering_]
    plt.figure(figsize=(8, 3))
    plt.plot(reachability)
    plt.title(title)
    plt.xlabel("Ordered points")
    plt.ylabel("Reachability distance")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    ensure_dirs(outpath.parent)
    plt.savefig(outpath, dpi=200)
    if show:
        plt.show()
    plt.close()


def step4_optics_compare() -> Path:
    out_root = RUN_DIR / "step4_optics_compare"
    ensure_dirs(out_root)

    for kind in KINDS:
        print("\n" + "=" * 90)
        print("[KIND]", kind)

        raw_out, opt_out, cmp_out = prepare_compare_dirs(out_root, kind)

        Xr, yr = load_step1_variant(kind, "raw")
        Xo, yo = load_step1_variant(kind, "optimized")

        # -------- GRID SEARCH RAW --------
        raw_rows = []
        best_raw = None
        best_raw_labels = None
        best_raw_model = None
        pur_tbl_raw_best = None

        for ms in OPTICS_MIN_SAMPLES_LIST:
            for xi in OPTICS_XI_LIST:
                for mcs in OPTICS_MIN_CLUSTER_SIZE_LIST:
                    model, labels = optics_fit_predict(Xr, ms, xi, mcs)
                    metrics, _ = eval_noise_clustering(Xr, yr, labels)
                    raw_rows.append({"min_samples": int(ms), "xi": float(xi), "min_cluster_size": float(mcs), **metrics})

        df_raw = pd.DataFrame(raw_rows).sort_values(["silhouette_excl_noise", "noise_ratio"], ascending=[False, True])
        df_raw.to_csv(raw_out / "tables" / "optics_grid_results.csv", index=False)

        best_raw = choose_best_by_target_k(df_raw, target_k=TARGET_K, max_noise_ratio=OPTICS_MAX_NOISE_RATIO)
        if best_raw is not None:
            best_raw_model, best_raw_labels = optics_fit_predict(
                Xr, best_raw["min_samples"], best_raw["xi"], best_raw["min_cluster_size"]
            )
            best_metrics_raw, pur_tbl_raw_best = eval_noise_clustering(Xr, yr, best_raw_labels)

            save_json({**best_raw, **best_metrics_raw}, raw_out / "meta" / "best_params.json")
            pd.DataFrame({"cluster": best_raw_labels}).to_csv(raw_out / "tables" / "labels_best.csv", index=False)
            pur_tbl_raw_best.to_csv(raw_out / "tables" / "purity_best.csv", index=False)

            plot_reachability(
                best_raw_model,
                title=f"OPTICS Reachability — RAW — {kind} (ms={best_raw['min_samples']}, xi={best_raw['xi']}, mcs={best_raw['min_cluster_size']})",
                outpath=raw_out / "figures" / "reachability_best.png",
            )
            plot_pca2_side_by_side_truth_vs_cluster(
                Xr, yr, best_raw_labels,
                title=f"RAW — {kind} — TRUE vs OPTICS",
                outpath=cmp_out / "figures" / "raw_truth_vs_optics.png",
                include_noise=True,
            )

        # -------- GRID SEARCH OPT --------
        opt_rows = []
        best_opt = None
        best_opt_labels = None
        best_opt_model = None
        pur_tbl_opt_best = None

        for ms in OPTICS_MIN_SAMPLES_LIST:
            for xi in OPTICS_XI_LIST:
                for mcs in OPTICS_MIN_CLUSTER_SIZE_LIST:
                    model, labels = optics_fit_predict(Xo, ms, xi, mcs)
                    metrics, _ = eval_noise_clustering(Xo, yo, labels)
                    opt_rows.append({"min_samples": int(ms), "xi": float(xi), "min_cluster_size": float(mcs), **metrics})

        df_opt = pd.DataFrame(opt_rows).sort_values(["silhouette_excl_noise", "noise_ratio"], ascending=[False, True])
        df_opt.to_csv(opt_out / "tables" / "optics_grid_results.csv", index=False)

        best_opt = choose_best_by_target_k(df_opt, target_k=TARGET_K, max_noise_ratio=OPTICS_MAX_NOISE_RATIO)
        if best_opt is not None:
            best_opt_model, best_opt_labels = optics_fit_predict(
                Xo, best_opt["min_samples"], best_opt["xi"], best_opt["min_cluster_size"]
            )
            best_metrics_opt, pur_tbl_opt_best = eval_noise_clustering(Xo, yo, best_opt_labels)

            save_json({**best_opt, **best_metrics_opt}, opt_out / "meta" / "best_params.json")
            pd.DataFrame({"cluster": best_opt_labels}).to_csv(opt_out / "tables" / "labels_best.csv", index=False)
            pur_tbl_opt_best.to_csv(opt_out / "tables" / "purity_best.csv", index=False)

            plot_reachability(
                best_opt_model,
                title=f"OPTICS Reachability — OPT — {kind} (ms={best_opt['min_samples']}, xi={best_opt['xi']}, mcs={best_opt['min_cluster_size']})",
                outpath=opt_out / "figures" / "reachability_best.png",
            )
            plot_pca2_side_by_side_truth_vs_cluster(
                Xo, yo, best_opt_labels,
                title=f"OPT — {kind} — TRUE vs OPTICS",
                outpath=cmp_out / "figures" / "opt_truth_vs_optics.png",
                include_noise=True,
            )

        # -------- SIDE-BY-SIDE tables --------
        top_raw = df_raw.head(10).reset_index(drop=True)
        top_opt = df_opt.head(10).reset_index(drop=True)
        print("[BEST RAW]", best_raw)
        print("[BEST OPT]", best_opt)
        side_by_side_tables(top_raw, top_opt, title_left=f"RAW Top-10 ({kind})", title_right=f"OPT Top-10 ({kind})")

        # -------- Side-by-side reachability in one figure --------
        fig, axes = plt.subplots(1, 2, figsize=(12, 3), sharey=True)
        if best_raw_model is not None:
            r = best_raw_model.reachability_[best_raw_model.ordering_]
            axes[0].plot(r)
            axes[0].set_title("RAW reachability")
            axes[0].grid(True, alpha=0.3)
        else:
            axes[0].set_title("RAW reachability (no best)")
            axes[0].axis("off")

        if best_opt_model is not None:
            r = best_opt_model.reachability_[best_opt_model.ordering_]
            axes[1].plot(r)
            axes[1].set_title("OPT reachability")
            axes[1].grid(True, alpha=0.3)
        else:
            axes[1].set_title("OPT reachability (no best)")
            axes[1].axis("off")

        fig.suptitle(f"OPTICS Reachability side-by-side — {kind}", y=1.05)
        fig.tight_layout()
        plt.show()
        fig.savefig(cmp_out / "figures" / "reachability_side_by_side.png", dpi=200)
        plt.close(fig)

        # Save compare summary
        comp = []
        if best_raw is not None:
            comp.append({"variant": "raw", **best_raw})
        if best_opt is not None:
            comp.append({"variant": "optimized", **best_opt})
        pd.DataFrame(comp).to_csv(cmp_out / "tables" / "best_compare.csv", index=False)

    print("\n[DONE] Step4 (OPTICS) completed. Saved under:", out_root)
    return out_root


# Run step4
step4_optics_compare()


## 9. Orchestrating the Full Pipeline

Finally, a single driver function executes the pipeline end-to-end: exporting (optional), preprocessing, and running the comparative experiments for K-Means, DBSCAN, and OPTICS. This entry-point standardizes execution and ensures that all outputs are produced under a timestamped run directory.

In [ ]:
def run_all():
    step0_export_to_csv()  # optional
    step1_prepare_raw_and_optimized()
    step2_kmeans_compare()
    step3_dbscan_compare()
    step4_optics_compare()

run_all()
